In [1]:
import numpy as np
import os
from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf

## Load data

In [3]:
DATASET_PATH = "./dataset/UCI HAR Dataset/"

In [4]:
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

In [5]:
def load_X(X_signals_paths):
    X_signals = []
    
    for signal_type_path in X_signals_paths:
        with open(signal_type_path, "r") as f:
            X_signals.append(
                [np.array(serie, dtype=np.float32)
                    for serie in [row.replace('  ', ' ').strip().split(' ') for row in f]]
            )
    return np.transpose(X_signals, (1, 2, 0))

def load_y(y_path):

    # Read dataset from disk, dealing with text file's syntax
    with open(y_path, "r") as f:
        y = np.array(
            [elem for elem in [
                row.replace('  ', ' ').strip().split(' ') for row in f
            ]], 
            dtype=np.int32
        )
            
    y = y.reshape(-1,)
    # Substract 1 to each output class for friendly 0-based indexing 
    return y - 1

In [6]:
X_train_signals_paths = [
    DATASET_PATH + "train/Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
X_test_signals_paths = [
    DATASET_PATH + "test/Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

In [7]:
har_X_train = load_X(X_train_signals_paths)
har_X_test = load_X(X_test_signals_paths)

In [8]:
y_train_path = DATASET_PATH + "train/y_train.txt"
y_test_path = DATASET_PATH + "test/y_test.txt"

In [9]:
har_y_train = load_y(y_train_path)
har_y_test = load_y(y_test_path)

## Prepare data

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(har_X_train, har_y_train, stratify=har_y_train, random_state=42)

In [11]:
X_train.shape, y_train.shape

((5514, 128, 9), (5514,))

In [12]:
X_train = np.transpose(X_train, (0, 2, 1))
X_valid = np.transpose(X_valid, (0, 2, 1))
har_X_test = np.transpose(har_X_test, (0, 2, 1))

In [13]:
X_train.shape, y_train.shape

((5514, 9, 128), (5514,))

# Build RNN model

## Hyperparameters

In [14]:
batch_size = 60
n_epochs = 100
learning_rate = 0.0001

seq_len = 128
n_channels = 9
n_neurons = 100
n_layers = 3

# outputs
n_classes = 6

## first RNN model

In [15]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, n_channels, seq_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [16]:
with tf.name_scope("rnn"):
    
    # Convolutional layers
    basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,
                                             activation=tf.nn.relu)
    outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
    
    # only one RNN cell, the shape of states is (None, n_neurons)
    #states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(states, n_classes)

In [17]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [18]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [19]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [20]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [21]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for X_batch, y_batch in get_batches(X_train, y_train, batch_size):
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
        valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        
        if epoch % 10 == 0:
            print("Epoch: ", epoch, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_first_rnn_model.ckpt")

Epoch:  0 Valid accuracy:  0.583787
Epoch:  10 Valid accuracy:  0.880849
Epoch:  20 Valid accuracy:  0.908596
Epoch:  30 Valid accuracy:  0.919478
Epoch:  40 Valid accuracy:  0.923286
Epoch:  50 Valid accuracy:  0.917301
Epoch:  60 Valid accuracy:  0.917845
Epoch:  70 Valid accuracy:  0.92111
Epoch:  80 Valid accuracy:  0.934168
Epoch:  90 Valid accuracy:  0.934712


In [22]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_first_rnn_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_first_rnn_model.ckpt
valid Accuracy:  0.935256
Test Accuracy:  0.856464


## LSTM model

In [23]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, n_channels, seq_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [24]:
with tf.name_scope("rnn"):
    
    # Convolutional layers
    lstm_cells = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons,
                                             activation=tf.nn.relu) 
                      for _ in range(n_layers)]
    multi_cell = tf.contrib.rnn.MultiRNNCell(lstm_cells)
    outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
    top_layer_h_state = states[-1][-1]
    states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(top_layer_h_state, n_classes)

In [25]:
top_layer_h_state.shape

TensorShape([Dimension(None), Dimension(100)])

In [26]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [27]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [28]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [29]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [30]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for X_batch, y_batch in get_batches(X_train, y_train, batch_size):
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
        valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        
        if epoch % 10 == 0:
            print("Epoch: ", epoch, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_lstm_model.ckpt")

Epoch:  0 Valid accuracy:  0.326442
Epoch:  10 Valid accuracy:  0.905332
Epoch:  20 Valid accuracy:  0.917845
Epoch:  30 Valid accuracy:  0.935256
Epoch:  40 Valid accuracy:  0.944505
Epoch:  50 Valid accuracy:  0.930359
Epoch:  60 Valid accuracy:  0.944505
Epoch:  70 Valid accuracy:  0.945593
Epoch:  80 Valid accuracy:  0.944505
Epoch:  90 Valid accuracy:  0.943417


In [31]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_lstm_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_lstm_model.ckpt
valid Accuracy:  0.947225
Test Accuracy:  0.892433
